In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
from pypfopt.expected_returns import ema_historical_return

In [3]:
from data import *

In [1]:
class dataset:
    data_path = '../data/SP500/SP_20180402_20200401.csv'

class train:
    start_date = '2019-08-02'
    end_date = '2019-09-01'

In [5]:
train_dataset = SP500_loader(dataset, train)

name,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZION,ZTS
name,,,,,,,,,,,,,,,,,,,,,
A,20.592345,8.123689,39.974549,42.929623,22.836407,24.156238,19.667871,31.720432,66.458387,30.531679,...,10.025579,45.110364,13.676764,12.962000,6.425283,12.524994,22.630757,24.699943,8.201541,27.193089
AAL,8.123689,3.204799,15.769977,16.935756,9.008973,9.529647,7.758984,12.513725,26.217864,12.044761,...,3.955095,17.796059,5.395489,5.113515,2.534777,4.941116,8.927844,9.744139,3.235512,10.727686
AAP,39.974549,15.769977,77.599930,83.336419,44.330795,46.892897,38.179929,61.576762,129.011242,59.269115,...,19.461988,87.569745,26.549791,25.162268,12.472973,24.313936,43.931582,47.948355,15.921105,52.788134
AAPL,42.929623,16.935756,83.336419,89.496972,47.607901,50.359402,41.002338,66.128756,138.548256,63.650519,...,20.900694,94.043242,28.512455,27.022361,13.395024,26.111317,47.179176,51.492884,17.098055,56.690439
ABBV,22.836407,9.008973,44.330795,47.607901,25.325015,26.788676,21.811187,35.177182,73.700724,33.858884,...,11.118121,50.026288,15.167196,14.374541,7.125481,13.889911,25.096956,27.391632,9.095308,30.156470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XYL,12.524994,4.941116,24.313936,26.111317,13.889911,14.692680,11.962696,19.293490,40.422345,18.570448,...,6.097912,27.437721,8.318692,7.883947,3.908084,7.618145,13.764828,15.023380,4.988468,16.539800
YUM,22.630757,8.927844,43.931582,47.179176,25.096956,26.547436,21.614771,34.860400,73.037025,33.553974,...,11.017999,49.575786,15.030610,14.245093,7.061314,13.764828,24.870950,27.144961,9.013402,29.884902
ZBH,24.699943,9.744139,47.948355,51.492884,27.391632,28.974733,23.591062,38.047772,79.714980,36.621896,...,12.025402,54.108621,16.404896,15.547557,7.706947,15.023380,27.144961,29.626892,9.837519,32.617351


In [6]:
mu = ema_historical_return(train_dataset)
C = train_dataset.cov()

In [7]:
N = len(train_dataset.columns)
K = 30

In [8]:
randoms = np.random.uniform(0, 1, (K))
randoms = list(randoms / randoms.sum())
w = np.random.permutation([*randoms, *[0 for i in range(N - K)]])
ws = w.copy()

gamma = 0.95
gamma_u = 0.95
u = 0.1
T = 100
I = int(N * (N - 1) / 2)

In [10]:
for k in range(2 * N):
    wu = w.copy()
    choices = np.where(w != 0.0)[0]
    i = np.random.choice(choices)
    j = np.random.randint(N)
    if w[j] == 0:
        wu[j] = w[i]
        wu[i] = 0
    else:
        ui = ru = np.random.uniform(0, min(u, w[i], w[j]))
        wu[j] = w[j] + ui
        wu[i] = w[i] - ui
    dl = L(wu, mu, C) - L(w, mu, C)
    if dl > 0:
        w = wu.copy()
        if L(w, mu, C) > L(ws, mu, C):
            ws = w.copy()
    else:
        p = np.random.uniform(0, 1)
        if p <= np.exp(dl / T):
            w = wu.copy()
    T = gamma * T
    u = gamma_u * u

In [11]:
ws

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0133049 , 0.        ,
       0.00625893, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03665734, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01290469, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.05803392,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05428702, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.017041  , 0.        , 0.     

In [13]:
from utils.financial_measures import *

In [17]:
weights, \
corr_min, \
corr_max, \
corr_mean, \
corr_std, \
port_return, \
port_std, \
port_sharpe, \
port_information, \
port_modigliani = calculate_measures(train_dataset.columns, train_dataset, ws)

In [20]:
port_sharpe

2.76711415900063

Index(['AKAM', 'ALGN', 'AMD', 'ARNC', 'AYI', 'BLL', 'CAG', 'CERN', 'CF', 'DRE',
       'F', 'FE', 'FITB', 'GWW', 'HES', 'INFO', 'IR', 'KMX', 'MAT', 'O', 'PHM',
       'RCL', 'REGN', 'SBUX', 'SO', 'SPG', 'UAL', 'USB', 'WEC', 'ZBH'],
      dtype='object', name='name')